In [6]:
# Before delving into the implementation of YOLOv8 for our Proof of Concept (PoC), 
# it's crucial to consider a few key aspects regarding the dataset provided by Spare-it. 
# There are notable characteristics we must address to ensure the success of our model.

# Firstly, our dataset exhibits class imbalance, where certain recyclable 
# classes are underrepresented or absent entirely. For instance, labels 
# like "paper towel" may occur frequently, while others may be scarce. 
# To mitigate this imbalance, we plan to leverage pretrained models. 
# These models not only help address class imbalances but also enhance the 
# overall accuracy of our model as Spare-it's dataset expands and evolves.

# Furthermore, within our dataset, we encounter a distinction between material 
# classes and specific names (e.g., "paper towel"). Notably, the material class 
# serves as a superclass encompassing various specific names. Therefore, achieving 
# optimal performance on these specific names is crucial for enhancing the overall model accuracy.

# Our initial step involves transforming Spare-it's dataset, currently in cocojson 
# format, to align with the requirements of YOLOv8's dataset. Initially, we prioritize 
# handling bounding boxes (bbox) and defer addressing segmentation aspects until we achieve satisfactory accuracy levels.

In [10]:
# You will need to have images folder with all the images and cocojson folder with all the labels to run codes below.
import json
import os

# Your directories
coco_dir = './cocojson'  # Directory containing COCO JSON files
labels_dir = './labels'  # Final directory to save the YOLO format files

# These are index mapping we have to do.
# YOLOv8 needs your name starting from 0 incrementing by 1.
# Spare-it has id values not in this format, so we have mapped them in this way.
id_to_index = {1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7, 10: 8, 11: 9, 
               14: 10, 15: 11, 16: 12, 17: 13, 18: 14, 19: 15, 115: 16, 114: 17, 
               116: 18, 117: 19, 122: 20, 123: 21, 124: 22, 127: 23, 20: 24, 21: 25, 
               22: 26, 23: 27, 24: 28, 25: 29, 26: 30, 27: 31, 30: 32, 125: 33, 126: 34, 
               36: 35, 38: 36, 40: 37, 43: 38, 44: 39, 45: 40, 46: 41, 47: 42, 48: 43, 
               49: 44, 50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 
               58: 53, 59: 54, 60: 55, 120: 56, 121: 57, 63: 58, 65: 59, 66: 60, 67: 61, 
               68: 62, 69: 63, 70: 64, 72: 65, 73: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 
               82: 72, 83: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 118: 79, 119: 80, 89: 81, 
               91: 82, 92: 83, 93: 84, 94: 85, 95: 86, 96: 87, 97: 88, 98: 89, 99: 90, 100: 91, 
               101: 92, 102: 93, 103: 94, 106: 95, 107: 96, 109: 97, 110: 98, 112: 99}

# Function to read Spare-it's custom COCO json into standardized YOLO format.
# I am basing this off from Ultralytics' JSON2YOLO.
# Reason building custom one is to have the id_to_index mapping within coco_to_yolo.
def coco_to_yolo(coco_json_path, labels_dir, id_to_index):
    with open(coco_json_path) as f:
        data = json.load(f)

    # We will need image_width, image_height, and overall annotation here
    for image in data['images']:
        image_width, image_height = image['width'], image['height']
        yolo_annotations = [] # our converted yolo data format

        for annotation in data['annotations']:
            category_id = id_to_index.get(annotation['category_id'], -1) # convert category_id using the id_to_index mapping
            x_min, y_min, bbox_width, bbox_height = annotation['bbox']
            x_center = x_min + (bbox_width / 2)
            y_center = y_min + (bbox_height / 2)

            x_center_normalized = x_center / image_width
            y_center_normalized = y_center / image_height
            width_normalized = bbox_width / image_width
            height_normalized = bbox_height / image_height

            yolo_format = f"{category_id} {x_center_normalized} {y_center_normalized} {width_normalized} {height_normalized}"
            yolo_annotations.append(yolo_format)

        base_filename = os.path.splitext(os.path.basename(coco_json_path))[0]
        output_filename = os.path.join(labels_dir, f"{base_filename}.txt")
        
        # Write output into YOLO format
        with open(output_filename, 'w') as f:
            for item in yolo_annotations:
                f.write("%s\n" % item)

# Creates labels directory and iterate over all cocojson files to that folder
def convert_directory(coco_dir, labels_dir, id_to_index):
    if not os.path.exists(labels_dir):
        os.makedirs(labels_dir)

    for filename in os.listdir(coco_dir):
        if filename.endswith(".json"):
            coco_json_path = os.path.join(coco_dir, filename)
            coco_to_yolo(coco_json_path, labels_dir, id_to_index)


convert_directory(coco_dir, labels_dir, id_to_index)

In [32]:
# Now we have all the data formatted to conduct training the model.
#
# We will need to use the dataset.yml, I have generated.
# Looks through the YOLO format id to get the label name and figure out where the train and val images are at.
# We will need to create a main folder called datasets. Inside this there will be train and val folders. Inside each
# of these three folder, we will have images and labels folder. By doing this you will be able to customize YOLOv8 model with own dataset.
import os
import shutil
import random

# Generate YOLO custom dataset format
subfolders = ['train', 'val']
for subfolder in subfolders:
    os.makedirs(os.path.join('./datasets', subfolder, 'images'), exist_ok=True)
    os.makedirs(os.path.join('./datasets', subfolder, 'labels'), exist_ok=True)

# Base dataset to create these splits
labels_path = './labels'
images_path = './images'
image_extensions = ['.jpg', '.jpeg', '.png', '.gif']

label_files = [file for file in os.listdir(labels_path) if file.endswith('.txt')]
random.shuffle(label_files)  # Randomize the list

# Calculate split sizes
# Change the percentange for your need
num_total = len(label_files)
num_train = int(num_total * 0.85)
num_val = num_total - num_train

# Split the files
train_files = label_files[:num_train]
val_files = label_files[num_train:]

def copy_files(files, source_path_images, source_path_labels, target_path_images, target_path_labels):
    for file in files:
        base_filename = os.path.splitext(file)[0]
        # Attempt to find and copy the first matching image file with any of the specified extensions
        copied = False
        for ext in image_extensions:
            image_file = base_filename + ext
            if os.path.exists(os.path.join(source_path_images, image_file)):
                shutil.copy(os.path.join(source_path_images, image_file),
                            os.path.join(target_path_images, image_file))
                copied = True
                break  # stops after first match
        if not copied:
            print(f"No matching image found for {file}")
        else:
            shutil.copy(os.path.join(source_path_labels, file), os.path.join(target_path_labels, file)) # Only add labels file if image exist.
            
# Copy files to respective directories
copy_files(train_files, images_path, labels_path, os.path.join('./datasets', 'train', 'images'), os.path.join('./datasets', 'train', 'labels'))
copy_files(val_files, images_path, labels_path, os.path.join('./datasets', 'val', 'images'), os.path.join('./datasets', 'val', 'labels'))

In [19]:
# Now you have to train using the Ultralytics
!pip install -q ultralytics

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [45]:
# https://docs.ultralytics.com/modes/train/
from ultralytics import YOLO
# model = YOLO('yolov8n.yaml')
# model = YOLO('yolov8n.pt')
# model = YOLO('yolov8n.yaml').load('yolov8n.pt') # build from YAML and transfer weights
results = model.train(
    data='dataset.yaml', # custom dataset
    epochs=100, # default 100 ~ higher epoch was worse went to like 38
    imgsz=640, # default 640 ~ imgsz didn't change the end result
    batch=25, # default 16,~8 was bad
    patience=9, # default 100
    dropout=0.0, # default 0.0 ~ anything on above 0.1 was bad
)

Ultralytics YOLOv8.1.37 🚀 Python-3.11.6 torch-2.1.2 CUDA:0 (Tesla V100S-PCIE-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=dataset.yaml, epochs=100, time=None, patience=9, batch=25, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train72232, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

train: Scanning /projectnb/spareit/jasonoh/datasets/train/labels.cache... 1720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1720/1720 [00:00<?, ?it/s]
val: Scanning /projectnb/spareit/jasonoh/datasets/val/labels.cache... 304 images, 0 backgrounds, 0 corrupt: 100%|██████████| 304/304 [00:00<?, ?it/s]


Plotting labels to runs/detect/train72232/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=9.6e-05, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005859375000000001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train72232
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      6.73G     0.9621      1.428      1.083        181        640: 100%|██████████| 69/69 [00:11<00:00,  5.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  6.50it/s]

                   all        304       2277       0.44      0.176      0.162      0.106


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     24/100      6.72G     0.9711      1.343      1.077        144        640: 100%|██████████| 69/69 [00:11<00:00,  5.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  6.53it/s]


                   all        304       2277      0.384      0.184      0.176      0.116
Stopping training early as no improvement observed in last 9 epochs. Best results observed at epoch 15, best model saved as best.pt.
To update EarlyStopping(patience=9) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

24 epochs completed in 0.091 hours.
Optimizer stripped from runs/detect/train72232/weights/last.pt, 6.9MB
Optimizer stripped from runs/detect/train72232/weights/best.pt, 6.9MB

Validating runs/detect/train72232/weights/best.pt...
Ultralytics YOLOv8.1.37 🚀 Python-3.11.6 torch-2.1.2 CUDA:0 (Tesla V100S-PCIE-32GB, 32501MiB)
YOLOv8n summary (fused): 168 layers, 3340724 parameters, 0 gradients, 9.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.90it/s]


                   all        304       2277      0.313        0.2      0.174      0.118
             Paper Cup        304         69      0.262       0.42      0.293      0.174
Snack or Candy Bag or Wrapper        304        195       0.25      0.456      0.279      0.172
             Wax Paper        304          3          1          0          0          0
          Latex Gloves        304          5          0          0     0.0456     0.0358
  Juice or Other Pouch        304          2          1          0      0.497      0.248
Facemask and Other PPE        304          1          1          0     0.0995     0.0697
   Shelf Stable Carton        304          3          0          0          0          0
        Soiled Plastic        304         38      0.128      0.237      0.113     0.0826
              Ceramics        304          1          0          0          0          0
        Unclassifiable        304         49      0.235      0.102     0.0952      0.041
            Fi

In [38]:
results = model("./images/Compost_000eeeeb-0444-4f74-91e3-d6329c6e9c85_ec3a2daf-29b4-4804-9216-e6a71b07a335_35addbdb-52aa-42ac-a9c7-26ed974484f7.jpeg")


image 1/1 /projectnb/spareit/jasonoh/images/Compost_000eeeeb-0444-4f74-91e3-d6329c6e9c85_ec3a2daf-29b4-4804-9216-e6a71b07a335_35addbdb-52aa-42ac-a9c7-26ed974484f7.jpeg: 1280x960 2 Snack or Candy Bag or Wrappers, 2 Compostable Fiber Wares, 6.8ms
Speed: 8.2ms preprocess, 6.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)


In [ ]:
# Export the model to ONNX format
success = model.export(format='onnx')

## Over Sampling methods with new data sets included